##### Fluid Dynamics-Inspired Emotional Analysis & Sentiment Flow for Argumentation Mining (AM)
- Stance basesd Argument Mining Modeling Using NRC and SneticNet Emotion Analysis


The idea is to use "Emotional Analysis" (NRC,SenticNet, SOCAL) to improve

- Stance Claissfication
- Fine-Grained Classifcation 

- https://aclanthology.org/J11-2001.pdf    (SO-CAL)
- https://github.com/sfu-discourse-lab/SO-CAL
- Original of SC 
    - https://www.cs.cornell.edu/home/llee/papers/sentiment.pdf

In [110]:
import shutil
import pandas as pd
from SentimentFlow import SpeechProcessor
from SentimentFlow import SentimentFlowCalculator
#https://github.com.mcas.ms/dpicca/SentimentFlow
pd.set_option("display.max_colwidth", None)
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import os
import spacy
from nrclex import NRCLex

import pandas as pd
import spacy
from nrclex import NRCLex
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder





# Argumentative Microtext Part 1

we've done a extensive pre-processing on microtext to extract all layers of annotations. The focus is stance classifcation 

In [103]:
df = pd.read_json("../data/dfMT-PC_SA_CP_topicID_AduType.json") 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 111
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EDU        112 non-null    object
 1   adu_type   112 non-null    object
 2   topic_id   89 non-null     object
 3   Relations  112 non-null    object
 4   Label      112 non-null    object
 5   STANCE     89 non-null     object
dtypes: object(6)
memory usage: 6.1+ KB


In [4]:
df[0:10]   

,EDU,adu_type,topic_id,Relations,Label,STANCE
0,"[Yes, it's annoying and cumbersome to separate your rubbish properly all the time., Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins., But still Germany produces way too much rubbish, and too many resources are lost when what actually should be separated and recycled is burnt., We Berliners should take the chance and become pioneers in waste separation!]","[opp, opp, pro, pro, pro]",waste_separation,"[[a1, a5, reb], [a2, a1, sup], [a3, c1, und], [a4, c3, add]]","[0, 0, 0, 0, 1]",pro
1,"[One can hardly move in Friedrichshain or Neukölln these days without permanently scanning the ground for dog dirt., And when bad luck does strike and you step into one of the many 'land mines' you have to painstakingly scrape the remains off your soles., Higher fines are therefore the right measure against negligent, lazy or simply thoughtless dog owners., Of course, first they'd actually need to be caught in the act by public order officers,, but once they have to dig into their pockets, their laziness will sure vanish!]","[pro, pro, pro, opp, pro]",higher_dog_poo_fines,"[[a1, a3, sup], [a2, a3, sup], [a4, a3, reb], [a5, c4, und]]","[0, 0, 1, 0, 0]",pro
2,"[Health insurance companies should not cover treatment in complementary medicine, unless the promised effect and its medical benefit have been concretely proven., Yet this very proof is lacking in most cases., Patients do often report relief of their complaints after such treatments., But as long as it is unclear as to how this works, the funds should rather be spent on therapies where one knows with certainty.]","[pro, opp, pro, opp, pro]",health_insurance_cover_complementary_medicine,"[[a2, a1, reb], [a3, a2, reb], [a4, a1, reb], [a5, c4, und]]","[1, 0, 0, 0, 0]",con
3,"[Of course there are a number of programmes in public broadcasting that are not worth the licencing fee,, and others, such as “Musikantenstadl” and soap operas, are only interesting to certain audiences., Nevertheless, everybody should contribute to the funding of the public broadcasters in equal measure,, for we need general and independent media., After all we want to get our view of the world neither through the lens of the government nor through that of rich media entrepreneurs.]","[opp, opp, pro, pro, pro]",public_broadcasting_fees_on_demand,"[[a1, a3, reb], [a2, a3, reb], [a4, a3, sup], [a5, a4, sup]]","[0, 0, 1, 0, 0]",con
4,"[Intelligence services must urgently be regulated more tightly by parliament;, this should be clear to everyone after the disclosures of Edward Snowden., Granted, those concern primarily the British and American intelligence services,, but the German services evidently do collaborate with them closely., Their tools, data and expertise have been used to keep us under surveillance for a long time.]","[pro, pro, opp, pro, pro]",stricter_regulation_of_intelligence_services,"[[a2, a1, sup], [a3, c2, und], [a4, c3, und], [a5, a4, sup]]","[1, 0, 0, 0, 0]",pro
5,"[The death penalty is a legal means that as such is not practicable in Germany., For one thing, inviolable human dignity is anchored in our constitution,, and furthermore no one may have the right to adjudicate upon the death of another human being., Even if many people think that a murderer has already decided on the life or death of another person,, this is precisely the crime that we should not repay with the same.]","[pro, pro, pro, opp, pro]",introduce_capital_punishment,"[[a2, a1, sup], [a3, a1, sup], [a4, a1, reb], [a5, c4, und]]","[1, 0, 0, 0, 0]",con
6,"[Dog dirt on the pavement requires my complete, ground-focused attention on my way to work., This results in a lack of alertness in the road traffic comparable to a minor offense of the traffic regulations., That's why causing such obstacles must to be more urgently punished., Dog owners may think that dog dirt is only a harmless secretion of an animal like any other., Yet then they misj

### A-NRC for emotion analysis

In [63]:
def calculate_nrc(text):
    """Calculate NRC sentiment frequencies for a given text."""
    if isinstance(text, list):  
        text = " ".join(text) 
    if not isinstance(text, str): 
        raise ValueError(f"Expected string input, but got {type(text)}")  
    nrc_sentiment = NRCLex(text)  
    return nrc_sentiment.affect_frequencies
#nrc_sentiment.raw_emotion_scores,  nrc_sentiment.top_emotions

In [64]:
print(df.EDU[0],"\n", df.adu_type[0])

["Yes, it's annoying and cumbersome to separate your rubbish properly all the time.", 'Three different bin bags stink away in the kitchen and have to be sorted into different wheelie bins.', 'But still Germany produces way too much rubbish', 'and too many resources are lost when what actually should be separated and recycled is burnt.', 'We Berliners should take the chance and become pioneers in waste separation!'] 
 ['opp', 'opp', 'pro', 'pro', 'pro']


In [ ]:
calculate_nrc(df.EDU[0][0])#which feaure are usefull for stance classfication= 

{'fear': 0.0,
 'anger': 0.14285714285714285,
 'anticip': 0.0,
 'trust': 0.0,
 'surprise': 0.0,
 'positive': 0.0,
 'negative': 0.42857142857142855,
 'sadness': 0.14285714285714285,
 'disgust': 0.14285714285714285,
 'joy': 0.0,
 'anticipation': 0.14285714285714285}

In [40]:
def calculate_nrc(text):
    """Calculate NRC sentiment frequencies for a given text."""
    if isinstance(text, list):  
        text = " ".join(text) 
    if not isinstance(text, str): 
        raise ValueError(f"Expected string input, but got {type(text)}")  
    nrc_sentiment = NRCLex(text)  
    return nrc_sentiment.affect_frequencies


In [69]:
def calculate_nrc(text):
    """Calculate NRC emotion frequencies for a given text."""
    nrc_sentiment = NRCLex(text)
    affect_dict = nrc_sentiment.affect_frequencies

    base_emotions = ['fear', 'anger', 'trust', 'surprise', 
                     'positive', 'negative', 'sadness', 'disgust', 'joy', "anticipation"]
    
    for emotion in base_emotions:
        if emotion not in affect_dict:
            affect_dict[emotion] = 0.0

    return {emotion: affect_dict[emotion] for emotion in base_emotions}

def process_data(df):
    """Process DataFrame by applying NRC analysis and filtering emotions."""
    df_exp = df.explode(['EDU', 'adu_type'])
    
    nrc_features = df_exp['EDU'].apply(lambda x: pd.Series(calculate_nrc(x)))
    
    return pd.concat([df_exp[['adu_type']], nrc_features], axis=1)

def visualize_correlation(processed_df):
    """Generate a heatmap visualization of NRC emotions by ADU type."""
    grouped = processed_df.groupby('adu_type').mean().reset_index()
    
    melted = grouped.melt(id_vars='adu_type', 
                          var_name='Emotion', 
                          value_name='Intensity')

    fig = px.imshow(
        grouped.set_index('adu_type').T,
        labels=dict(x="ADU Type", y="NRC Dimension", color="Intensity"),
        color_continuous_scale='RdBu',
        title="NRC Emotion Intensity by ADU Type",
        aspect="auto"
    )
    
    fig.update_layout(
        xaxis_title="ADU Type",
        yaxis_title="NRC Emotion Dimension",
        coloraxis_colorbar=dict(title="Intensity")
    )
    
    return fig

processed_df = process_data(df)
fig = visualize_correlation(processed_df)
fig.show()


### B-Senticnet for emotion analysis

# Argumentative Microtext Part 2

In [111]:
df2 = pd.read_json("../data/dfMT2-PC_SA_CP_topicID_AduType.json") 

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 170
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   EDU        171 non-null    object
 1   adu_type   171 non-null    object
 2   topic_id   171 non-null    object
 3   Relations  171 non-null    object
 4   Label      171 non-null    object
 5   STANCE     171 non-null    object
dtypes: object(6)
memory usage: 9.4+ KB


In [135]:
def classify_pro_opp_RF(df):
    df_exp = df.explode(['EDU', 'adu_type'])

    X = df_exp['EDU']
    y = df_exp['adu_type']

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y) 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    clf.fit(X_train_tfidf, y_train)
    
    y_pred_rf = clf.predict(X_test_tfidf)
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    class_report_rf = classification_report(y_test, y_pred_rf)

    print(f"\nRandom Forest Accuracy: {accuracy_rf}")
    print(f"Random Forest Classification Report:\n{class_report_rf}")
    


    majority_classifier = DummyClassifier(strategy='most_frequent')
    majority_classifier.fit(X_train_tfidf, y_train)
    
    y_pred_majority = majority_classifier.predict(X_test_tfidf)
    accuracy_majority = accuracy_score(y_test, y_pred_majority)
    class_report_majority = classification_report(y_test, y_pred_majority)

    print(f"\nMajority Classifier Accuracy: {accuracy_majority}")
    print(f"Majority Classifier Classification Report:\n{class_report_majority}")
    

    return accuracy_rf, class_report_rf, accuracy_majority, class_report_majority




In [134]:
accuracy_rf, class_report_rf, accuracy_majority, class_report_majority = classify_pro_opp_RF(df)



Random Forest Accuracy: 0.8017241379310345
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.09      0.15        22
           1       0.82      0.97      0.89        94

    accuracy                           0.80       116
   macro avg       0.61      0.53      0.52       116
weighted avg       0.74      0.80      0.75       116


Majority Classifier Accuracy: 0.8103448275862069
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.81      1.00      0.90        94

    accuracy                           0.81       116
   macro avg       0.41      0.50      0.45       116
weighted avg       0.66      0.81      0.73       116



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [120]:
def handle_mismatched_rows(df):
    mismatched_rows = df[df.apply(lambda row: len(row['EDU']) != len(row['adu_type']), axis=1)]
    
    #if not mismatched_rows.empty:
    #    print(f"Found {len(mismatched_rows)} rows with mismatched list lengths.")
    #    print(mismatched_rows)
    
    df = df[df.apply(lambda row: len(row['EDU']) == len(row['adu_type']), axis=1)]
    
    return df

In [136]:
df2_new = handle_mismatched_rows(df2)

In [137]:
accuracy_rf_2, class_report_rf_2, accuracy_majority_2, class_report_majority_2 = classify_pro_opp_RF(df2_new)



Random Forest Accuracy: 0.85
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.86      0.99      0.92       137

    accuracy                           0.85       160
   macro avg       0.43      0.50      0.46       160
weighted avg       0.73      0.85      0.79       160


Majority Classifier Accuracy: 0.85625
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.86      1.00      0.92       137

    accuracy                           0.86       160
   macro avg       0.43      0.50      0.46       160
weighted avg       0.73      0.86      0.79       160



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# test SVM model

In [138]:
from sklearn.svm import SVC


def classify_pro_opp_SVM(df):
    df = handle_mismatched_rows(df)
    
    df_exp = df.explode(['EDU', 'adu_type'])

    X = df_exp['EDU']
    y = df_exp['adu_type']

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y) 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    clf = SVC(kernel='linear', class_weight='balanced', random_state=42)
    clf.fit(X_train_tfidf, y_train)
    
    y_pred_svm = clf.predict(X_test_tfidf)
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    class_report_svm = classification_report(y_test, y_pred_svm)

    print(f"\nSVM Accuracy: {accuracy_svm}")
    print(f"SVM Classification Report:\n{class_report_svm}")

    majority_classifier = DummyClassifier(strategy='most_frequent')
    majority_classifier.fit(X_train_tfidf, y_train)
    
    y_pred_majority = majority_classifier.predict(X_test_tfidf)
    accuracy_majority = accuracy_score(y_test, y_pred_majority)
    class_report_majority = classification_report(y_test, y_pred_majority)

    print(f"\nMajority Classifier Accuracy: {accuracy_majority}")
    print(f"Majority Classifier Classification Report:\n{class_report_majority}")

    return accuracy_svm, class_report_svm, accuracy_majority, class_report_majority


In [139]:
accuracy_svm, class_report_svm, accuracy_majority, class_report_majority = classify_pro_opp_SVM(df)


SVM Accuracy: 0.7672413793103449
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.32      0.34        22
           1       0.85      0.87      0.86        94

    accuracy                           0.77       116
   macro avg       0.61      0.60      0.60       116
weighted avg       0.75      0.77      0.76       116


Majority Classifier Accuracy: 0.8103448275862069
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.81      1.00      0.90        94

    accuracy                           0.81       116
   macro avg       0.41      0.50      0.45       116
weighted avg       0.66      0.81      0.73       116



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [140]:
accuracy_svm_2, class_report_svm_2, accuracy_majority_2, class_report_majority_2 = classify_pro_opp_SVM(df2)


SVM Accuracy: 0.79375
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.26      0.27        23
           1       0.88      0.88      0.88       137

    accuracy                           0.79       160
   macro avg       0.57      0.57      0.57       160
weighted avg       0.79      0.79      0.79       160


Majority Classifier Accuracy: 0.85625
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.86      1.00      0.92       137

    accuracy                           0.86       160
   macro avg       0.43      0.50      0.46       160
weighted avg       0.73      0.86      0.79       160



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# ADD NRC values as features

In [142]:
def classify_pro_opp_SVM_NRC(df):
    df = handle_mismatched_rows(df)
    
    df_exp = df.explode(['EDU', 'adu_type'])

    X = df_exp['EDU']
    y = df_exp['adu_type']

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    nrc_features = X.apply(lambda text: calculate_nrc(text))
    nrc_df = pd.DataFrame(nrc_features.tolist())  
    
    vectorizer = TfidfVectorizer(max_features=5000)
    X_tfidf = vectorizer.fit_transform(X)
    
    X_combined = np.hstack((X_tfidf.toarray(), nrc_df.to_numpy()))
    
    X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)

    clf = SVC(kernel='linear', class_weight='balanced', random_state=42)
    clf.fit(X_train, y_train)
    
    y_pred_svm = clf.predict(X_test)
    accuracy_svm = accuracy_score(y_test, y_pred_svm)
    class_report_svm = classification_report(y_test, y_pred_svm)

    print(f"\nSVM Accuracy: {accuracy_svm}")
    print(f"SVM Classification Report:\n{class_report_svm}")

    majority_classifier = DummyClassifier(strategy='most_frequent')
    majority_classifier.fit(X_train, y_train)
    
    y_pred_majority = majority_classifier.predict(X_test)
    accuracy_majority = accuracy_score(y_test, y_pred_majority)
    class_report_majority = classification_report(y_test, y_pred_majority)

    print(f"\nMajority Classifier Accuracy: {accuracy_majority}")
    print(f"Majority Classifier Classification Report:\n{class_report_majority}")

    return accuracy_svm, class_report_svm, accuracy_majority, class_report_majority

In [143]:
accuracy_svm_nrc, class_report_svm_nrc, accuracy_majority_nrc, class_report_majority_nrc = classify_pro_opp_SVM_NRC(df)


SVM Accuracy: 0.8103448275862069
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.32      0.39        22
           1       0.85      0.93      0.89        94

    accuracy                           0.81       116
   macro avg       0.68      0.62      0.64       116
weighted avg       0.79      0.81      0.79       116


Majority Classifier Accuracy: 0.8103448275862069
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.81      1.00      0.90        94

    accuracy                           0.81       116
   macro avg       0.41      0.50      0.45       116
weighted avg       0.66      0.81      0.73       116



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [144]:
accuracy_svm_nrc2, class_report_svm_nrc2, accuracy_majority_nrc2, class_report_majority_nrc2 = classify_pro_opp_SVM_NRC(df2)


SVM Accuracy: 0.81875
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.26      0.29        23
           1       0.88      0.91      0.90       137

    accuracy                           0.82       160
   macro avg       0.61      0.59      0.59       160
weighted avg       0.80      0.82      0.81       160


Majority Classifier Accuracy: 0.85625
Majority Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.86      1.00      0.92       137

    accuracy                           0.86       160
   macro avg       0.43      0.50      0.46       160
weighted avg       0.73      0.86      0.79       160



h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

h:\moha\NLP\AM\SA\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

